In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
#import cupy as cp
from tqdm import tqdm
tqdm.pandas()

In [3]:
class CFG:
    exp_no = "topics-context-for-retriever_v4-reranker" # 毎回変える＆確認する事！
    # Load Paths: Common(基本変えなくていい)
    # competition_data
    compdata_topics = "/content/drive/MyDrive/Colab_Files/kaggle/lecr/input/00_competition/topics.csv"
    compdata_content = "/content/drive/MyDrive/Colab_Files/kaggle/lecr/input/00_competition/content.csv"
    compdata_correlations = "/content/drive/MyDrive/Colab_Files/kaggle/lecr/input/00_competition/correlations.csv"
    compdata_sample_sub = "/content/drive/MyDrive/Colab_Files/kaggle/lecr/input/00_competition/sample_submission.csv"

    # Save paths（動的に変わるので基本いじらなくていい）
    # data save path
    data_save_path = f"/content/drive/MyDrive/Colab_Files/kaggle/lecr/input/01_for_retriever/{exp_no}"

print("data_path: ", CFG.data_save_path)
! mkdir -p {CFG.data_save_path}

data_path:  /content/drive/MyDrive/Colab_Files/kaggle/lecr/input/01_for_retriever/topics-context-for-retriever_v4-reranker


In [4]:
# Load data
print('Load data')
topics = pd.read_csv(CFG.compdata_topics)
correlations = pd.read_csv(CFG.compdata_correlations)

# fillna
print('fillna')
topics['title'] = topics['title'].fillna("")
topics['description'] = topics['description'].fillna("")

# cut length
print('cut length')
topics['title'] = topics['title'].map(lambda x: ' '.join(x.split(' ')[:17]))
topics['description'] = topics['description'].map(lambda x: ' '.join(x.split(' ')[:31]))

Load data
fillna
cut length


In [5]:
# =========================================================================================
# Utils
# =========================================================================================
import pickle

def show_df(df:pd.DataFrame, rows_=5):
    print(df.shape)
    return display(df.head(rows_))

def pickle_dump(obj_, filename):
    with open(f'{filename}', 'wb') as f:
        pickle.dump(obj_, f)
        
def pickle_load(path_):
    with open(f'{path_}', 'rb') as f:
        obj_ = pickle.load(f)
    return obj_

def create_list_batch(list_:list, batch_size:int):
    out_ = []
    for i in range(len(list_)):
        batch_ = list_[i*batch_size:(i+1)*batch_size]
        if len(batch_) == 0:
            pass
        else:
            out_.append(batch_)
    return out_

# =========================================================================================
# context関連
# =========================================================================================
def get_parent(id_:str, arr_set):
    return arr_set["parent_id_arr"][arr_set["topics_id_arr"] == id_][0]

def get_title(id_:str, arr_set):
    return arr_set["title_id_arr"][arr_set["topics_id_arr"] == id_][0]

def get_desc(id_:str, arr_set):
    return arr_set["desc_id_arr"][arr_set["topics_id_arr"] == id_][0]

def search_parent_id(topics_id:str, arr_set):
    parents = []
    for i in range(20):
        if i == 0:
            parent_id = get_parent(topics_id, arr_set)
        else: 
            parent_id = get_parent(parent_id, arr_set)

        if pd.isna(parent_id):
            break
        else:    
            parents.append(parent_id)
    return parents

def search_parent_text(topics_id:str, arr_set):
    parents = []
    for i in range(20):
        if i == 0:
            parent_id = get_parent(topics_id, arr_set)
        else: 
            parent_id = get_parent(parent_id, arr_set)

        if pd.isna(parent_id):
            break
        else:    
            parents.append(get_title(parent_id, arr_set))
    return parents

def search_parent_text_split(topics_id:str, arr_set):
    parents = []
    for i in range(20):
        if i == 0:
            parent_id = get_parent(topics_id, arr_set)
        else: 
            parent_id = get_parent(parent_id, arr_set)

        if pd.isna(parent_id):
            break
        else:    
            parents.append(get_title(parent_id, arr_set))
    if len(parents) > 0:
        return sum([i.split(' ') for i in parents],  [])
    else: 
        return []    

def search_parent_title_joined(topics_id:str, arr_set, sep='<|=t_sep=|>'):
    parents = []
    for i in range(20):
        if i == 0:
            parent_id = get_parent(topics_id, arr_set)
        else: 
            parent_id = get_parent(parent_id, arr_set)

        if pd.isna(parent_id):
            break
        else:    
            parents.append(get_title(parent_id, arr_set))
    return sep.join(parents)

def search_parent_desc_joined(topics_id:str, arr_set, sep='<|=t_sep=|>'):
    parents = []
    for i in range(20):
        if i == 0:
            parent_id = get_parent(topics_id, arr_set)
        else: 
            parent_id = get_parent(parent_id, arr_set)

        if pd.isna(parent_id):
            break
        else:    
            parents.append(get_desc(parent_id, arr_set))
    return sep.join(parents)

In [6]:
# create context
print('create context')
arr_set = {}
arr_set["topics_id_arr"] = topics['id'].to_numpy()
arr_set["parent_id_arr"] = topics['parent'].to_numpy()
arr_set["title_id_arr"] = topics['title'].to_numpy()
arr_set["desc_id_arr"] = topics['description'].to_numpy()

create context


In [7]:
# debug
topics_tmp = topics.head(100).copy()
topics_tmp['context_title_str'] = topics_tmp['id'].progress_map(lambda x: search_parent_title_joined(x, arr_set))
topics_tmp['context_desc_str'] = topics_tmp['id'].progress_map(lambda x: search_parent_desc_joined(x, arr_set))


100%|██████████| 100/100 [00:04<00:00, 22.80it/s]


In [8]:
#topics['context_ids'] = topics['id'].progress_map(lambda x: search_parent_id(x, arr_set))

In [9]:
topics['context_title_str'] = topics['id'].progress_map(lambda x: search_parent_title_joined(x, arr_set))
topics['context_title_str'] = topics['context_title_str'].fillna("")

100%|██████████| 76972/76972 [28:16<00:00, 45.37it/s]


In [10]:
topics['context_desc_str'] = topics['id'].progress_map(lambda x: search_parent_desc_joined(x, arr_set))
topics['context_desc_str'] = topics['context_desc_str'].fillna("")

100%|██████████| 76972/76972 [27:39<00:00, 46.37it/s]


In [11]:
topics.head()

,id,title,description,channel,category,level,language,parent,has_content,context_title_str,context_desc_str
0,t_00004da3a1b2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Открития и проекти<|=t_sep=|>Физика<|=t_sep=|>...,Преоткрийте най-важните научни наблюдения в хо...
1,t_000095e03056,Unit 3.3 Enlargements and Similarities,,b3f329,aligned,2,en,t_aa32fb6252dc,False,Junior High Level 3<|=t_sep=|>Ghana JHS Curric...,<|=t_sep=|>
2,t_00068291e9a4,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True,Álgebra: funções<|=t_sep=|>9º Ano<|=t_sep=|>Ma...,Nessa unidade temática vamos trabalhar com fun...
3,t_00069b63a70a,Transcripts,,6e3ba4,source,3,en,t_4054df11a74e,True,Flow Charts: Logical Thinking?<|=t_sep=|>Engin...,This lesson is focused on flow charts. It supp...
4,t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...,000cf7,source,4,bg,t_e2452e21d252,True,Показателни и логаритмични функции<|=t_sep=|>А...,Тази тема покрива:\n\n- Корени и рационални ст...


In [12]:
print('get children')
sep='<|=t_sep=|>'
topics['children_id'] = topics['id'].map(topics.groupby('parent')['id'].apply(list))
topics['children_title'] = topics['id'].map(topics.groupby('parent')['title'].apply(list))
topics['children_title'] = topics['children_title'].fillna("")
topics['children_title'] = topics['children_title'].map(lambda x: x if x=='' else sep.join(x))

get children


In [13]:
topics_out = topics[['id', 'context_title_str', 'context_desc_str', 'children_title']]
pickle_dump(topics_out, f"{CFG.data_save_path}/topics_with_context.pkl")

In [14]:
topics_load = pickle_load(f"{CFG.data_save_path}/topics_with_context.pkl")

In [15]:
topics_load.head()

,id,context_title_str,context_desc_str,children_title
0,t_00004da3a1b2,Открития и проекти<|=t_sep=|>Физика<|=t_sep=|>...,Преоткрийте най-важните научни наблюдения в хо...,
1,t_000095e03056,Junior High Level 3<|=t_sep=|>Ghana JHS Curric...,<|=t_sep=|>,Objective 3.3.4 Identify an object and its ima...
2,t_00068291e9a4,Álgebra: funções<|=t_sep=|>9º Ano<|=t_sep=|>Ma...,Nessa unidade temática vamos trabalhar com fun...,
3,t_00069b63a70a,Flow Charts: Logical Thinking?<|=t_sep=|>Engin...,This lesson is focused on flow charts. It supp...,
4,t_0006d41a73a8,Показателни и логаритмични функции<|=t_sep=|>А...,Тази тема покрива:\n\n- Корени и рационални ст...,


In [16]:
print(f'{CFG.data_save_path}/topics_with_context.pkl')

/content/drive/MyDrive/Colab_Files/kaggle/lecr/input/01_for_retriever/topics-context-for-retriever_v4-reranker/topics_with_context.pkl
